<a href="https://colab.research.google.com/github/arsh1207/BigDataProject/blob/master/User_based_CF_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from pyspark.rdd import RDD
from pyspark.sql import Row
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import dask.array as da
from numpy import cov
from numpy import std
from sklearn.metrics.pairwise import cosine_similarity
import random
import math

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.0.0-preview2/spark-3.0.0-preview2-bin-hadoop2.7.tgz
!tar -xvf spark-3.0.0-preview2-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-preview2-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

# User User based Collaborative filtering

In [0]:
def init_spark():
    spark = SparkSession \
        .builder \
        .config("spark.driver.memory", "15g") \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()
    return spark

In [0]:
spark = init_spark()

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [51]:
!ls "/content/drive/My Drive/Colab Notebooks"

 anime.csv					   rating.csv
 Bike_Sharing.ipynb				   test_data_animes.csv
 Classifier_Interpretability_Project.ipynb	   test_data_user.csv
 Content_based_recommendation_system_Colab.ipynb   testfile2
'Copy of Part1_TensorFlow.ipynb'		   training_data.csv
'Copy of Part2_Music_Generation.ipynb'		   Untitled0.ipynb
 predicted					   Untitled1.ipynb
 predicted1					   User_based_CF_colab.ipynb


In [0]:
# Reading Data from anime.csv
df2 = pd.read_csv('/content/drive/My Drive/Colab Notebooks/anime.csv')

In [0]:
# Redaing from rating.csv and transforming data
df = spark.read.text("/content/drive/My Drive/Colab Notebooks/rating.csv").rdd
data = df.map(lambda x: x[0].split(','))
data2 = data.groupBy(lambda x : x[0])

In [0]:
data3 = data2.collect()

In [0]:
# Preparing data : a dictionary utility = {users: {anime : ratings}}
utility = {}
all_animes = []
all_users = []
for user in range(len(data3)):
    if data3[user][0] == 'user_id':
        continue
    utility[int(data3[user][0])] = {}
    #if int(data3[user][0]) not in all_users:
    #    all_users.append(int(i[0]))
    for i in data3[user][1]:        
        if int(i[1]) not in all_animes:
            all_animes.append(int(i[1]))
        if int(i[2]) == -1:
            continue            
        utility[int(i[0])][int(i[1])] = int(i[2])

In [0]:
all_users = list(utility.keys())

In [0]:
def sim2(matrix, user1, user2):
    #This method returns the similarity between two users
    #It uses Pearson Correlation Coefficient for similarity calculation
    matched = []
    for anime in matrix[user1].keys():
        if anime in matrix[user2].keys():
            matched.append(anime) 
    x = []
    y = []    
    for anime in matched:
        x.append(matrix[user1][anime])
        y.append(matrix[user2][anime])        

    f = (lambda i, j: i - j)
    x = f(x, np.mean(x))
    y = f(y, np.mean(y))
    # Pearson Correlation Coefficient formula below
    numerator = (lambda x, y : x*y)
    denominator = (lambda x: x*x)
    d = math.sqrt(sum(denominator(x))) * math.sqrt(sum(denominator(y)))
    if d == 0:
        return 0
    similarity = sum(numerator(x, y))/ d
    return similarity

In [0]:
def most_similar_user(matrix, user, n=500):
    # This method returna a list of n most similar users
    similarity = {}
    for p in matrix.keys():
        similarity[p] = sim2(matrix, user, p)
    sorted_list = sorted(similarity.items(), key = lambda kv:(kv[1], kv[0]))
    return sorted_list[-n:]     

In [0]:
def recommend(matrix, user):
    # this method returns the predictions wrt most similar users
    similar_users = most_similar_user(matrix, user)
    predictions = {}
    for anime in all_animes:
        num = 0
        den = 0
        for user_ in similar_users:
            if anime in matrix[user_[0]].keys():
                num += matrix[user_[0]][anime] * user_[1]
                den += user_[1]
        if den == 0:
            continue
        #print(matrix[user[0]])
        #if anime in matrix[user[0]].keys():
        predictions[anime] = float(num/den)
    return predictions

In [0]:
# Creating Test data of randomly selecting 1000 users
test= int(0.20 * len(all_users))
test_data_user = []
for _ in range(1000):
    random_user = random.randint(1, len(all_users))
    if all_users[random_user] not in test_data_user:
        test_data_user.append(all_users[random_user])

In [0]:
# loading the same test data
test_data_users = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test_data_user.csv")
test_data_animes = pd.read_csv("/content/drive/My Drive/Colab Notebooks/test_data_animes.csv")
test_data_user = test_data_users["user_id"].to_list()
test_data_animes = test_data_animes['test_animes'].to_list()

In [50]:
print(len(test_data_animes))

3304


In [0]:
# Creating test data of ramdomly selecting 30% of the total animes list
test_animes = int(0.2 * len(all_animes))
test_data_animes = []
for _ in range(test_animes):
    random_anime = random.randint(1, len(all_animes))
    if all_animes[random_anime] not in test_data_animes:
        test_data_animes.append(all_users[random_anime])

In [0]:
# making a deep copy of the data 
import copy
training_data = copy.deepcopy(utility)

In [0]:
#Creating training data by deleting the test data from the original data
for user in test_data_user:
    for anime in test_data_animes:
        if anime in training_data[user].keys():        
            del training_data[user][anime]                        

In [0]:
#Saving the test data of 1000 users
#list_to_df = pd.DataFrame(test_data_user, columns =['user_id'])
#list_to_df.to_csv (r'/content/drive/My Drive/Colab Notebooks/test_data_user.csv', index = False, header=True)

In [0]:
def start():
    predicted = {}
    counter = 0
    for user in test_data_user:
        predicted[user] = recommend(training_data, user)
        counter += 1
        if counter%50 ==0:
          print(counter)
    return predicted

In [34]:

predicted1 = start()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [0]:
def RMSE(actual, predicted):
    error = 0
    n = 0
    for user in test_data_user:
        for anime in test_data_animes:
            if anime in predicted[user].keys() and anime in actual[user].keys():
                error += (actual[user][anime] - predicted[user][anime])**2
                n += 1
    return math.sqrt(error/n)

In [47]:
RMSE(utility, predicted1)

1.7525965512669202

## Applying the common practice 

In [0]:
avg_rating_of_user = {}
global_avg = 0
counter = 0
for user in all_users:
    count = 0
    summ = 0
    for anime in utility[user].keys():
        summ += utility[user][anime]
        count += 1
        global_avg += utility[user][anime]
        counter += 1
    if count != 0:
        avg_rating_of_user[user] = summ/count
    else:
        avg_rating_of_user[user] = 0
global_avg = global_avg/counter

In [40]:
global_avg

7.808496893124035

In [0]:
avg_rating_of_animes = {}
for anime in all_animes:
    count = 0
    summ = 0
    for user in all_users:
        if anime in utility[user].keys():
            summ += utility[user][anime]
            count += 1
    if count != 0:
        avg_rating_of_animes[anime] = summ/count
    else:
        avg_rating_of_animes[anime] = 0
    

In [0]:
def recommend_with_regularization(matrix, user):
    # this method returns the predictions wrt most similar users
    similar_users = most_similar_user(matrix, user)
    predictions = {}
    for anime in all_animes:
        num = 0
        den = 0
        for user_ in similar_users:
            if anime in matrix[user_[0]].keys():
                b_xj = global_avg + (avg_rating_of_user[user_[0]] - global_avg) + (avg_rating_of_animes[anime]- global_avg)
                num += (matrix[user_[0]][anime]- b_xj) * user_[1]
                den += user_[1]
        if den == 0:
            continue
        # if anime in matrix[user[0]].keys():
        b_xi = global_avg + (avg_rating_of_user[user] - global_avg) + (avg_rating_of_animes[anime] - global_avg)
        if b_xi + float(num/den) > 10:
            predictions[anime] = 10
        else:
            predictions[anime] = b_xi + float(num/den)
    return predictions

In [0]:
def start2():
    predicted = {}
    counter = 0
    for user in test_data_user:
        predicted[user] = recommend_with_regularization(training_data, user)
        counter += 1
        if counter%50 ==0:
            print(counter)
    return predicted

In [44]:
predicted = start2()

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950


In [55]:
RMSE(utility, predicted)

1.4800267481453462

In [0]:
# Saving the predictions
import pickle
file_Name = "/content/drive/My Drive/Colab Notebooks/predicted"

with open(file_Name, 'wb') as handle:
    pickle.dump(predicted, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [0]:
#Loading the saved predicted files
file_Name = "/content/drive/My Drive/Colab Notebooks/testfile2"
with open(file_Name, 'rb') as handle:
    predicted3 = pickle.load(handle)

#predicted == b


In [0]:
def recommend_animes(for_user, n = 10):
    predictions = recommend(training_data, for_user)
    sorted_list = sorted(predictions.items(), key = lambda kv:(kv[1], kv[0]))
    for anime in sorted_list[-n:]:
        values = df2[['anime_id','name', 'genre', 'type']].loc[df2['anime_id']==anime[0]]
        print(str(values['anime_id']) + "," + str(values['name']) + "," + str(values['genre']) + "," + str(values['type']) + "\n")
    

In [0]:
recommend_animes(3)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


113    30503
Name: anime_id, dtype: int64,113    Noragami Aragoto
Name: name, dtype: object,113    Action, Adventure, Shounen, Supernatural
Name: genre, dtype: object,113    TV
Name: type, dtype: object

1817    30991
Name: anime_id, dtype: int64,1817    Himouto! Umaru-chan: Umaru-chan Mou Ikkai!
Name: name, dtype: object,1817    Comedy, School, Seinen, Slice of Life
Name: genre, dtype: object,1817    OVA
Name: type, dtype: object

4954    31121
Name: anime_id, dtype: int64,4954    Monster Musume no Iru Nichijou: Hobo Mainichi ...
Name: name, dtype: object,4954    Comedy, Slice of Life
Name: genre, dtype: object,4954    ONA
Name: type, dtype: object

369    31173
Name: anime_id, dtype: int64,369    Akagami no Shirayuki-hime 2nd Season
Name: name, dtype: object,369    Drama, Fantasy, Romance, Shoujo
Name: genre, dtype: object,369    TV
Name: type, dtype: object

3375    31234
Name: anime_id, dtype: int64,3375    Himouto! Umaru-chanS
Name: name, dtype: object,3375    Comedy, Seinen, Slic

In [0]:
def recommend_animes_regularized(for_user, n = 10):
    predictions = recommend_with_regularization(training_data, for_user)
    sorted_list = sorted(predictions.items(), key = lambda kv:(kv[1], kv[0]))
    for anime in sorted_list[-n:]:
        values = df2[['anime_id','name', 'genre', 'type']].loc[df2['anime_id']==anime[0]]
        print(str(values['anime_id']) + "," + str(values['name']) + "," + str(values['genre']) + "," + str(values['type']) + "\n")
    

In [0]:
recommend_animes_regularized(3)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


1156    1488
Name: anime_id, dtype: int64,1156    Area 88
Name: name, dtype: object,1156    Action, Adventure, Drama, Military, Romance
Name: genre, dtype: object,1156    OVA
Name: type, dtype: object

1546    1577
Name: anime_id, dtype: int64,1546    Taiho Shichau zo
Name: name, dtype: object,1546    Action, Comedy, Police, Seinen
Name: genre, dtype: object,1546    OVA
Name: type, dtype: object

2172    1901
Name: anime_id, dtype: int64,2172    11-nin Iru!
Name: name, dtype: object,2172    Action, Adventure, Drama, Mystery, Romance, Sc...
Name: genre, dtype: object,2172    Movie
Name: type, dtype: object

3259    1951
Name: anime_id, dtype: int64,3259    Manie-Manie: Meikyuu Monogatari
Name: name, dtype: object,3259    Adventure, Fantasy, Horror, Sci-Fi, Supernatural
Name: genre, dtype: object,3259    Movie
Name: type, dtype: object

1835    2182
Name: anime_id, dtype: int64,1835    Robot Carnival
Name: name, dtype: object,1835    Fantasy, Sci-Fi
Name: genre, dtype: object,1835    OVA